In [1]:
import numpy as np
import matplotlib.pyplot as plt
from filters import sobel_filter, gaussian_filter, non_max_suppression
 
from matplotlib import pyplot
from matplotlib.patches import Circle, ConnectionPatch

from timeit import default_timer as timer

import imageIO.readwrite as IORW
import imageProcessing.pixelops as IPPixelOps
import imageProcessing.utilities as IPUtils
import imageProcessing.smoothing as IPSmooth



CORNERNESS_THRESHOLD = int('1'.replace(',','')) # set it here like this so we can easily change it
SEARCH_SIZE = 3//2 # first number is the window X by X size


In [2]:
filename_left_image = "./images/panoramaStitching/tongariro_left_01.png"
filename_right_image = "./images/panoramaStitching/tongariro_right_01.png"

(image_width, image_height, px_array_left_original)  = IORW.readRGBImageAndConvertToGreyscalePixelArray(filename_left_image)
(image_width, image_height, px_array_right_original) = IORW.readRGBImageAndConvertToGreyscalePixelArray(filename_right_image)

start = timer()
px_array_left = IPSmooth.computeGaussianAveraging3x3(px_array_left_original, image_width, image_height)
px_array_right = IPSmooth.computeGaussianAveraging3x3(px_array_right_original, image_width, image_height)
end = timer()
print("elapsed time image smoothing: ", end - start)

# make sure greyscale image is stretched to full 8 bit intensity range of 0 to 255
px_array_left = IPPixelOps.scaleTo0And255AndQuantize(px_array_left, image_width, image_height)
px_array_right = IPPixelOps.scaleTo0And255AndQuantize(px_array_right, image_width, image_height)

im = px_array_left

read image width=1000, height=750
read image width=1000, height=750
elapsed time image smoothing:  1.8673804
before scaling, min value = 0, max value = 249.2229720776
before scaling, min value = 0, max value = 252.21425735919996


In [3]:
alpha=0.04
blur_kernal_size=3
nms_size=3
num_of_corners=1000

In [4]:
im = np.array(im)

#Calculate Ix and Iy (sobel filter over im) - Stephen
Ix, Iy = sobel_filter(im)

#Calculate Ix^2, Iy^2 and IxIy (these are element wise multiplications not mat mult) - Stephen

Ix2 = np.square(Ix)
Iy2 = np.square(Iy)
IxIy = np.multiply(Ix, Iy)

#Gaussian blur on Ix^2, Iy^2 and IxIy - Kelham
Ix2 = gaussian_filter(Ix2, blur_kernal_size)
Iy2 = gaussian_filter(Iy2, blur_kernal_size)
IxIy = gaussian_filter(IxIy, blur_kernal_size)

#Computer corner score of every pixel - Kelham
cornerness_score = np.multiply(Ix, Iy) - np.square(IxIy) - alpha * np.square(IxIy)

In [5]:
#Threshold cornerness score - Zak
thresholded_array = np.zeros_like(cornerness_score)
for (x,y), point in np.ndenumerate(cornerness_score):
    if abs(point) >= CORNERNESS_THRESHOLD:
        thresholded_array[x,y] = point




In [6]:
thresholded_array

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -6.03613623e+07, -2.30808919e+07, ...,
        -1.81087598e+04,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.53719386e+07, -9.72799319e+06, ...,
        -6.05912427e+03,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00, -3.80143629e+06, -2.01697573e+07, ...,
        -1.41337419e+01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [7]:
#Perform non-maximal supression - Zak
suppressed_array = non_max_suppression(thresholded_array, SEARCH_SIZE)


In [8]:
suppressed_array

array([[       0.        ,        0.        ,        0.        , ...,
               0.        ,        0.        ,        0.        ],
       [       0.        ,        0.        ,        0.        , ...,
               0.        ,        0.        ,        0.        ],
       [       0.        ,        0.        , -9727993.19434159, ...,
               0.        ,        0.        ,        0.        ],
       ...,
       [       0.        ,        0.        ,        0.        , ...,
               0.        ,        0.        ,        0.        ],
       [       0.        ,        0.        ,        0.        , ...,
               0.        ,        0.        ,        0.        ],
       [       0.        ,        0.        ,        0.        , ...,
               0.        ,        0.        ,        0.        ]])

In [14]:
#Put remaining corner points into a list in the form (x, y) - Zak
score_feature_list = [] # optional list with the score included
score_feature_list = [(score, coords[0], coords[1]) for coords, score in list(np.ndenumerate(suppressed_array))]
score_feature_list.sort()

xy_feature_list = [(x,y) for (_,x,y) in score_feature_list]

#Return the num_of_corners strongest (highest corner score) corners from this lists - Zak
print(xy_feature_list[:100])

[(732, 221), (251, 791), (696, 100), (694, 102), (339, 926), (691, 94), (520, 118), (407, 82), (731, 222), (252, 790), (354, 371), (735, 219), (703, 478), (404, 81), (702, 479), (404, 102), (734, 220), (745, 137), (516, 118), (704, 134), (113, 996), (743, 140), (595, 204), (610, 233), (683, 196), (690, 42), (253, 789), (408, 81), (574, 50), (455, 51), (713, 279), (454, 52), (691, 393), (574, 53), (619, 284), (406, 84), (745, 399), (740, 264), (736, 256), (412, 107), (573, 41), (631, 128), (145, 819), (640, 85), (733, 221), (709, 228), (333, 886), (251, 792), (696, 403), (572, 42), (451, 55), (596, 140), (692, 392), (453, 53), (452, 54), (705, 87), (696, 101), (402, 77), (684, 195), (149, 906), (603, 336), (450, 56), (321, 887), (405, 85), (456, 50), (355, 371), (591, 85), (717, 332), (538, 113), (448, 4), (695, 102), (687, 2), (659, 253), (522, 69), (394, 62), (334, 396), (337, 83), (111, 990), (256, 795), (729, 193), (518, 74), (250, 793), (401, 250), (470, 37), (129, 919), (710, 227)

In [12]:
score_feature_list

[((0, 0), 0.0),
 ((0, 1), 0.0),
 ((0, 2), 0.0),
 ((0, 3), 0.0),
 ((0, 4), 0.0),
 ((0, 5), 0.0),
 ((0, 6), 0.0),
 ((0, 7), 0.0),
 ((0, 8), 0.0),
 ((0, 9), 0.0),
 ((0, 10), 0.0),
 ((0, 11), 0.0),
 ((0, 12), 0.0),
 ((0, 13), 0.0),
 ((0, 14), 0.0),
 ((0, 15), 0.0),
 ((0, 16), 0.0),
 ((0, 17), 0.0),
 ((0, 18), 0.0),
 ((0, 19), 0.0),
 ((0, 20), 0.0),
 ((0, 21), 0.0),
 ((0, 22), 0.0),
 ((0, 23), 0.0),
 ((0, 24), 0.0),
 ((0, 25), 0.0),
 ((0, 26), 0.0),
 ((0, 27), 0.0),
 ((0, 28), 0.0),
 ((0, 29), 0.0),
 ((0, 30), 0.0),
 ((0, 31), 0.0),
 ((0, 32), 0.0),
 ((0, 33), 0.0),
 ((0, 34), 0.0),
 ((0, 35), 0.0),
 ((0, 36), 0.0),
 ((0, 37), 0.0),
 ((0, 38), 0.0),
 ((0, 39), 0.0),
 ((0, 40), 0.0),
 ((0, 41), 0.0),
 ((0, 42), 0.0),
 ((0, 43), 0.0),
 ((0, 44), 0.0),
 ((0, 45), 0.0),
 ((0, 46), 0.0),
 ((0, 47), 0.0),
 ((0, 48), 0.0),
 ((0, 49), 0.0),
 ((0, 50), 0.0),
 ((0, 51), 0.0),
 ((0, 52), 0.0),
 ((0, 53), 0.0),
 ((0, 54), 0.0),
 ((0, 55), 0.0),
 ((0, 56), 0.0),
 ((0, 57), 0.0),
 ((0, 58), 0.0),
 ((0, 5